In [9]:
import numpy as np
import torch
import torch.utils.data as data
import pandas as pd
from src.error_measures import get_accuracy
from src.models import TwoLayerPerceptron
from src.train_model import training_loop

In [10]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [11]:
# setting seeds
np.random.seed(445)
torch.manual_seed(445)

## Data

In [12]:
data06 = pd.read_csv("../data/processed/target06.csv")

In [13]:
all_data = data.TensorDataset(torch.from_numpy((data06.values[:,:-1] - data06.values[:,:-1].min(0)) / data06.values[:,:-1].ptp(0)).float(), torch.from_numpy(data06.values[:,-1]).float())  # with normalization
train_dataset, test_dataset, valid_dataset = torch.utils.data.random_split(all_data, (round(0.7 * len(all_data)), round(0.2 * len(all_data)), round(0.1 * len(all_data))))

In [14]:
train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=False)
valid_loader = data.DataLoader(valid_dataset, batch_size=256, shuffle=False, drop_last=False)

## Prepare model

In [15]:
model = TwoLayerPerceptron(num_inputs=8, num_hidden=500, num_outputs=1)
print(model)

TwoLayerPerceptron(
  (lin1): Linear(in_features=8, out_features=500, bias=True)
  (bn1): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (lin2): Linear(in_features=500, out_features=1, bias=True)
)


In [16]:
training_loop(model, train_loader, valid_loader, device, num_epochs=50)

100%|██████████| 50/50 [00:25<00:00,  1.93it/s]

Accuracy on validation dataset: 39.653179190751445%


In [18]:
get_accuracy(model, test_loader, device=device, pct_close=0.2)

0.41734104046242776